In [1]:
! pip install -q python-dotenv
! pip install -q neo4j
! pip install -q langchain
! pip install -q langchain-openai
! pip install -q tiktoken


[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import dotenv
import os

dotenv.load_dotenv()

os.environ["NEO4J_URI"] = os.getenv("URL")
os.environ["NEO4J_USERNAME"]= os.getenv("USERNAME")
os.environ["NEO4J_PASSWORD"] = os.getenv("PASSWORD2")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAIKEY")

In [3]:
from langchain_community.graphs import Neo4jGraph
import sys
sys.path.append('../utils')  
from helper import write_chunks_to_df


In [4]:
graphDB = Neo4jGraph()

In [6]:
#graphDB.query("SHOW INDEXES")

In [7]:
#
# Initialize Semantic Vector Index
#
# DOCS: https://neo4j.com/docs/cypher-manual/current/indexes/semantic-indexes/vector-indexes/
#

vector_index_query=""" 
CALL db.index.vector.createNodeIndex(
  'accreditation_index',
  'Chunk',
  'embedding',
   1536,
  'cosine'
)
"""

graphDB.query(vector_index_query)

[]

In [8]:
graphDB.query("SHOW INDEXES")

[{'id': 5,
  'name': 'accreditation_index',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'VECTOR',
  'entityType': 'NODE',
  'labelsOrTypes': ['Chunk'],
  'properties': ['embedding'],
  'indexProvider': 'vector-1.0',
  'owningConstraint': None,
  'lastRead': None,
  'readCount': 0},
 {'id': 1,
  'name': 'index_343aff4e',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'LOOKUP',
  'entityType': 'NODE',
  'labelsOrTypes': None,
  'properties': None,
  'indexProvider': 'token-lookup-1.0',
  'owningConstraint': None,
  'lastRead': neo4j.time.DateTime(2024, 4, 17, 5, 29, 23, 62000000, tzinfo=<UTC>),
  'readCount': 802},
 {'id': 2,
  'name': 'index_f7700477',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'LOOKUP',
  'entityType': 'RELATIONSHIP',
  'labelsOrTypes': None,
  'properties': None,
  'indexProvider': 'token-lookup-1.0',
  'owningConstraint': None,
  'lastRead': None,
  'readCount': 0},
 {'id': 3,
  'name': 'unique_node',
  'state': 'ONLIN

### Generate Vector Embeddings

In [9]:
#
#  Query All Chunks without embedding
#

all_chunks_query = """ 
MATCH (c:Chunk) 
WHERE c.embedding IS null OR c.embedding = 0
RETURN c
"""


# attach embedding to chunk.embedding
vector_to_chunk_query = """ 
MATCH (c:Chunk {UUID: $UUID})
SET c.embedding = $vector
"""

In [10]:
result = graphDB.query(all_chunks_query)

In [11]:
#
# Generate Vectors for Chunk.text and update Chunk.embedding
#

chunk_dataframe = write_chunks_to_df(result)

In [13]:
#chunk_dataframe.head()
chunk_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      208 non-null    object
 1   nodeType  208 non-null    object
 2   UUID      208 non-null    object
dtypes: object(3)
memory usage: 5.0+ KB


In [14]:
#
# Chunk embedding function
# 
# DOCS: https://platform.openai.com/docs/guides/embeddings/use-cases
#


from openai import OpenAI

client =OpenAI()

#MODEL =  "text-embedding-3-small"
MODEL = "text-embedding-ada-002"

def get_embedding(text, model = MODEL):

    text = text.replace("\n"," ")

    return client.embeddings.create(input=[text], model=model).data[0].embedding

In [15]:
chunk_dataframe['vector'] = chunk_dataframe['text'].apply(lambda x:get_embedding(x))

In [16]:
chunk_dataframe.head()

,text,nodeType,UUID,vector
0,standard 1 strategic planning1.1 school mainta...,DATACHUNK,fb8549d4-efa5-4c55-a7d3-87522a416b98,"[0.002811488462612033, 0.0041966927237808704, ..."
1,progress planned strategies expected outcomes ...,DATACHUNK,d6e18b0a-2aca-4de8-a294-e6160f186a0d,"[-0.01016693189740181, 0.0034439638257026672, ..."
2,1.1 maintenance strategic plan school mission ...,DATACHUNK,a67bc7ec-d5f2-44c4-a6e0-6d1e94af611b,"[-0.008482272736728191, -0.010247484780848026,..."
3,plan . strategic plan developed refined engage...,DATACHUNK,e143576e-9893-4999-a5f7-a898d3388193,"[0.0022559347562491894, 0.0026812339201569557,..."
4,serve including level degree programs school o...,DATACHUNK,eecdf569-2c10-4574-ab4a-a8f6117a0b6d,"[0.007008944638073444, 0.008684996515512466, -..."


In [17]:
for i, row in chunk_dataframe.iterrows():

    graphDB.query(vector_to_chunk_query, 
                params ={
                    'UUID':row['UUID'],
                    'vector':row['vector']
                })

### Queries

In [21]:
semantic_index_query = """

CALL db.index.vector.queryNodes('accreditation_index', 2, $inputVector)
YIELD node AS responseNode, score

RETURN responseNode.text, score 
"""

In [31]:
query_text = "what are aacsb standard"
query_vector = get_embedding(query_text)
query_result = graphDB.query(semantic_index_query, 
                                params={
                                    "inputVector":query_vector
                                })

In [32]:
query_result

[{'responseNode.text': '3.1 faculty sufficiency school adopts applies criteria documenting faculty members participating supporting consistent mission . school adapt guidance particular situation mission developing implementing criteria indicate school meeting spirit intent standard . criteria address activities required attain participating supporting status depth breadth activities expected within typical aacsb accreditation review cycle maintain participating supporting status . criteria periodically reviewed',
  'score': 0.9091740846633911},
 {'responseNode.text': 'attain participating supporting status depth breadth activities expected within typical aacsb accreditation review cycle maintain participating supporting status . criteria periodically reviewed reflect focus continuous improvement . normally participating faculty members deliver least 75 percent school teaching globally i.e . across entire accredited unit participating faculty members deliver least 60 percent teaching w

In [26]:
#
# Standard QA
#
from langchain_community.vectorstores import Neo4jVector

VECTOR_INDEX_NAME = 'accreditiation-index'
VECTOR_NODE_LABEL = 'Chunk'
VECTOR_SOURCE_PROPERTY = 'text'
VECTOR_EMBEDDING_PROPERTY='embedding'


In [ ]:
#https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever/


In [28]:
# Query the knowledge graph in a RAG application
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate



graphDB.refresh_schema()

cypher_chain = GraphCypherQAChain.from_llm(
    graph=graphDB,
    cypher_llm=ChatOpenAI(temperature=0, model="gpt-4"),
    qa_llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo"),
    validate_cypher=True, # Validate relationship directions
    verbose=True
)

In [30]:
#cypher_chain.invoke({"query": "How many learning objectives are assessed"})
#cypher_chain.invoke({"query": "Which student learning goals were identified"}) # does not know
#cypher_chain.invoke({"query": "What are the descriptions of the Learning goal"}) ## does not know , see still Learning goal example
cypher_chain.invoke({"query": "Tell me about AACSB standards"}) ## does not know , see still Learning goal example




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (s:Standard {source: 'AACSB'}) RETURN s
Full Context:
[]

> Finished chain.


{'query': 'Tell me about AACSB standards',
 'result': "I don't know the answer."}

Out of the box LLM genreate cypher queries are not producing any results, OR are producing errors. @TODO, provide some examples to the mode an oppopriate queries based on the schema. 